In [1]:
# importing module
from pymongo import MongoClient
client = MongoClient("mongodb://localhost:27017/")

In [2]:
mydatabase = client['mongodb']

In [3]:
collection_names = mydatabase.list_collection_names()

In [4]:
collection = mydatabase['film']
cur = collection.aggregate([{'$count': "amount of movies"}])
cur.next()

{'amount of movies': 1000}